In [ ]:
# Make sure we have plotly & ipywidgets (!pip ... -q for quiet install)
# (note. need to restart jupyter if pip install triggered)
try:
    import plotly
except:
    !pip install plotly
try:
    import impywidgets
except:
    !pip install ipywidgets

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import IPython
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.offline as pyo
from ipywidgets import Output, VBox, Layout

In [ ]:
# Load in the data & do some minor column changes
try:
    jsonpath = os.path.join(os.environ.get("HOME"), "data", os.listdir("data")[0], "ntu.json")
except IOError:
    print("File not found in usual DAFNI location.")
    print("Try CWD, ", os.path.join(os.getcwd(), "ntu.json"))
    jsonpath = os.path.join(os.getcwd(), "ntu.json")
df = pd.read_json(jsonpath)
# Create datetime objects in table & create anim frames.
df['time'] =  pd.to_datetime(df[7])
df['year'] = df['time'].apply(lambda row: str(row.year))
def setanim(x):
    return int(x)-1939
df['frame']=df['year'].apply(lambda row: setanim(row))
# Re-order dataframe by date
df.sort_values(by='time', inplace = True)
#df

In [ ]:
ax = df[2].plot.hist(bins=30, range=[0, 16000])
plt.axvline(x=7174.0, color="orange")
ax.set_xlabel("Cargo (GRT)")
ax.set_ylabel("Ships Lost")
plt.show()

In [ ]:
pio.renderers.default = 'iframe'

# Setup a colour index by year
colourindex = dict(zip(map(str, range(1939, 1946)), 
                      ["lightblue", "yellow", "orange", "red", "violet", "darkgrey", "grey"]))

fig1 = px.scatter_mapbox(lat=df[9], lon=df[10], hover_name=df[1], hover_data={"Sunk": df[0], "Nationality": df[3], "Tonnage": df[2]}, 
                        color_discrete_sequence=px.colors.qualitative.Alphabet,color=df['year'],
                        color_discrete_map=colourindex, animation_frame=df["year"],
                        zoom=3, labels={"color": "Year"})
fig1.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ])
fig1.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 0})

In [ ]:
def openurl(url):
    IPython.display.display(IPython.display.Javascript('window.open("{url}");'.format(url=url)))

figw = go.FigureWidget(layout={'hovermode': 'closest'})
mapi = figw.add_scattergeo( lat = df[9],lon = df[10], mode="markers", text=df[1], marker=dict(color="red", opacity=0.4) )
                       

mapi.update_layout(title = 'Test links')
mapi.update_layout(
    title = '<span style="font-size: 14px;">Ships Lost to U-Boat Campaign (\'39-\'45)</span>' +
               '<i><span style="font-size: 12px;">   Data are clickable</span></i>',
    mapbox_style="white-bg",
    mapbox_layers=[
        { 
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        }
      ]
)
mapi.update_layout(margin={"r":0,"t":30,"l":0,"b":0})

out = Output()
@out.capture(clear_output=True)
def handle_click(trace, points, state):
    if points.point_inds:
        ind = points.point_inds[0]
        url = "https://www.uboat.net" + df[11].iloc[ind]
        #print("URL is", url)
        openurl(url)
        
mapi.data[0].on_click(handle_click)
VBox([out, figw])

In [ ]:
freq1 = df[3].value_counts().tolist()
freq2 = df[3].value_counts().index.tolist()

fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.6, 0.4],
    row_heights=[0.4, 0.6],
    specs=[[{"type": "scattergeo", "rowspan": 2}, {"type": "bar"}],
           [            None                    , {"type": "histogram"}]])

# Add globe
fig.add_trace(
    go.Scattergeo(lat=df[9],
                  lon=df[10],
                  mode="markers",
                  hoverinfo="text",
                  showlegend=False,
                  marker=dict(color="red", size=4, opacity=0.5)),
    row=1, col=1
)

# Add national losses
fig.add_trace(
    go.Bar(x=freq2[:12], y=freq1[:12], marker=dict(color="orange"), showlegend=False),
    row=1, col=2
)

# Add time plot
fig.add_trace(
    go.Histogram(x=df[7], y=df[2], histfunc="sum", marker=dict(color="darkblue"), showlegend=False),
    row=2, col=2
)
fig.update_yaxes(title_text="National Ship Losses", row=1, col=2)
fig.update_yaxes(title_text="Tonnage Lost <i>(GRT)</i>", row=2, col=2)

# Update geo subplot properties
fig.update_geos(
    projection_type="orthographic",
    landcolor="lightyellow",
    oceancolor="lightblue",
    showocean=True,
    lakecolor="LightBlue"
)

# Rotate x-axis labels
fig.update_xaxes(tickangle=45)

# Set theme, margin, and annotation in layout
fig.update_layout(
    template="seaborn",
    margin=dict(r=10, t=25, b=40, l=60),
    annotations=[
        dict(
            text="Source: uboat.net",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0)
    ]
)
# Add images
flags = {"uk": [0.715, 0.93, "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Flag_of_the_United_Kingdom_%281-2%29.svg/383px-Flag_of_the_United_Kingdom_%281-2%29.svg.png"],
         "us": [0.725, 0.87, "https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/1920px-Flag_of_the_United_States.svg.png"],
         "nw": [0.74, 0.80, "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Flag_of_Norway.svg/1280px-Flag_of_Norway.svg.png"],
         "nl": [0.77, 0.74, "https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Flag_of_the_Netherlands.svg/1920px-Flag_of_the_Netherlands.svg.png"],
         "gr": [0.81, 0.80, "https://upload.wikimedia.org/wikipedia/commons/thumb/5/5c/Flag_of_Greece.svg/1920px-Flag_of_Greece.svg.png"],
         "sj": [0.84, 0.74, "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Flag_of_the_Soviet_Union.svg/1920px-Flag_of_the_Soviet_Union.svg.png"]
        }
for key in flags.keys():
    fig.add_layout_image(
        dict(
            source=flags[key][2],
            x=flags[key][0],
            y=flags[key][1]
        ))
fig.update_layout_images(dict(
        xref="paper",
        yref="paper",
        sizex=0.04,
        sizey=0.3,
        xanchor="right",
        yanchor="bottom"
))

fig.show()